In [2]:
from datetime import date
import time

import numpy as np
import pandas as pd

from scipy.sparse import coo_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [159]:
MIN_USERS_OVERLAP_COEFF = 0.5   # The minimum percentage of co-read books between current_user and overlapped_user to be used to create a recommendation
MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF = 2   # The minimum number of books read and rated by current_user before the MIN_USERS_OVERLAP_COEFF factor will be used to filter overlapped_users
SIMILAR_USERS_NO = 15   # The number of similar users whose book preferences will be used to recommend books to the current_user
BOOKS_NO_TO_RECOMMEND = 10   # The number of books that will be recommended to the user based on his/her book list
MIN_USER_RATINGS_NO_CFB = 100   # The minimum number of ratings per user required to use user data in the recommender system
MIN_BOOK_RATINGS_NO_CFB = 3   # The minimum number of ratings for a single book to be used to build a Collaborative Filtering Based Recommender System
DEFAULT_RATING = 5   # The neutral rating of the book
MIN_BOOKS_NO_GOOD_RESULTS = 3    # The minimum number of books the user likes for which good results are obtained

## Functions

In [3]:
def calc_weighted_rating(row, avg_rating, num_of_ratings, min_thres, default_rating):
    """
    Calculates the weighted rating for a book based on its average rating, number of ratings, and a minimum threshold.

    Args:
        row (pd.Series): A row from the DataFrame containing the book's data.
        avg_rating (str): The column name for the average rating of the book.
        num_of_ratings (str): The column name for the number of ratings the book has received.
        min_thres (int): The minimum threshold for the number of ratings to be considered for the weighted rating.
        default_rating (float): The neutral rating of the book.

    Returns:
        float: The calculated weighted rating for the book.
    """
    
    weighted_rating = ((row[avg_rating] * row[num_of_ratings]) + 
      (min_thres * default_rating))/(row[num_of_ratings] + min_thres)
    return weighted_rating

In [4]:
def make_clickable(val, link_title="Amazon Image"):
    """
    Converts a URL into a clickable HTML link.

    Args:
        val (str): The URL to be converted into a clickable link.
        link_title (str, optional): The text to be displayed for the clickable link. Default is "Amazon Image".
        
    Returns:
        str: An HTML string that represents a clickable link.
    """
    
    return '<a target="_blank" href="{}">{}</a>'.format(val, val, link_title)


def show_image(val):
    """
    Converts a URL into an HTML string that displays an image with a clickable link.

    Args:
        val (str): The URL of the image to be displayed and linked.

    Returns:
        str: An HTML string that represents an image which is clickable, linking to the image URL.
    """
    
    return '<a href="{}"><img src="{}" width=50></img></a>'.format(val, val)

## Read datasets

In [61]:
books_df = pd.read_csv('../data/db_data/Books.csv')
users_df = pd.read_csv('../data/db_data/Users.csv')
ratings_df = pd.read_csv('../data/db_data/Ratings.csv')

print(f"books_df.shape = {books_df.shape}")
print(f"users_df.shape = {users_df.shape}")
print(f"ratings_df.shape = {ratings_df.shape}")

books_df.shape = (271358, 6)
users_df.shape = (278858, 3)
ratings_df.shape = (1031136, 3)


In [6]:
books_df.head(3)

,isbn,title,author,publication_year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


In [7]:
users_df.head(3)

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [8]:
ratings_df.head(3)

,user_id,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


## Find users with similar book preferences as the current user

In [218]:
books_user_like = [
    '0590353403',   # Harry Potter and the Sorcerer's Stone (Book 1)
    '0439064872',   # Harry Potter and the Chamber of Secrets (Book 2)
    '0345350499'    # The Mists of Avalon
]

books_user_like_titles = [
    "Harry Potter and the Sorcerer's Stone (Book 1)", 
    "Harry Potter and the Chamber of Secrets (Book 2)",
    "The Mists of Avalon"
]
ratings = [10, 9, 10]

In [171]:
books_user_like = [
    '0590353403',   # Harry Potter and the Sorcerer's Stone (Book 1)
    '0439064872'   # Harry Potter and the Chamber of Secrets (Book 2)
]

books_user_like_titles = [
    "Harry Potter and the Sorcerer's Stone (Book 1)",
    "Harry Potter and the Chamber of Secrets (Book 2)"
]

ratings = [10, 9]

In [219]:
current_user_id = -1

books_user_like_dict = {
    'user_id': [current_user_id for _ in range(len(books_user_like))],
    'isbn': books_user_like,
    'title': books_user_like_titles,
    'rating': ratings
}

books_user_like_df = pd.DataFrame.from_dict(books_user_like_dict)
books_user_like_df

,user_id,isbn,title,rating
0,-1,0590353403,Harry Potter and the Sorcerer's Stone (Book 1),10
1,-1,0439064872,Harry Potter and the Chamber of Secrets (Book 2),9
2,-1,0345350499,The Mists of Avalon,10


In [220]:
books_user_like_set = set(books_user_like_df['isbn'])   # Because theoretically, a user could give several ratings to a book with the same title
print(f"books_user_like_set = {books_user_like_set}")

books_user_like_set = {'0590353403', '0345350499', '0439064872'}


In [223]:
SIMILAR_USERS_NO = 15
if len(books_user_like_set) < MIN_BOOKS_NO_GOOD_RESULTS:
    SIMILAR_USERS_NO = 30

### Filter users

In [12]:
# Filter users in order to reduce overlap users search time
# Leave only users with more than MIN_USER_RATINGS_NO_CFB ratings
user_rating_count_df = (
    ratings_df
    .groupby('user_id')
    ['rating']
    .count()
    .reset_index()
    .rename(columns={'rating': 'user_ratings_no'})
)

print(f"ratings_df['user_id'].nunique() = {ratings_df['user_id'].nunique()}")
print(f"user_rating_count_df.shape = {user_rating_count_df.shape}\n")

print(
    f"len(user_rating_count_df[user_rating_count_df['user_ratings_no'] >= MIN_USER_RATINGS_NO_CFB]) = "
    + f"{len(user_rating_count_df[user_rating_count_df['user_ratings_no'] >= MIN_USER_RATINGS_NO_CFB])}"
)

user_rating_count_df = user_rating_count_df[user_rating_count_df['user_ratings_no'] >= MIN_USER_RATINGS_NO_CFB]
print(f"user_rating_count_df.shape = {user_rating_count_df.shape}\n")
user_rating_count_df.head(5)

ratings_df['user_id'].nunique() = 92106
user_rating_count_df.shape = (92106, 2)

len(user_rating_count_df[user_rating_count_df['user_ratings_no'] >= MIN_USER_RATINGS_NO_CFB]) = 1659
user_rating_count_df.shape = (1659, 2)



,user_id,user_ratings_no
87,254,300
161,507,129
265,882,124
437,1424,132
441,1435,118


In [13]:
print(f"ratings_df.shape before filtering by user: {ratings_df.shape}")
ratings_df = ratings_df.merge(user_rating_count_df, on='user_id', how='inner')
ratings_df.drop('user_ratings_no', inplace=True, axis=1)
print(f"ratings_df.shape after filtering by user: {ratings_df.shape}")

ratings_df.shape before filtering by user: (1031136, 3)
ratings_df.shape after filtering by user: (592996, 3)


### Filter books

**Filtering books without necessarily saving user-liked books may not be the best idea, because if a user likes a book with a low number of ratings, it is quite possible that it will be filtered out.**

In [16]:
# Filter users in order to reduce overlap users search time
# Leave books current_user liked and books with more than MIN_BOOK_RATINGS_NO_CFB ratings
start_time = time.time()

book_rating_count_df = (
    ratings_df
    .groupby('isbn')
    ['rating']
    .count()
    .reset_index()
    .rename(columns={'rating': 'book_ratings_no'})
)

print(f"book_rating_count_df.shape before book filtering = {book_rating_count_df.shape}")
book_filter_condition_result = (
    book_rating_count_df
    .apply(
        lambda row: (row['isbn'] in books_user_like_set) or (row['book_ratings_no'] >= MIN_BOOK_RATINGS_NO_CFB), 
        axis=1
    )
)
book_rating_count_df = book_rating_count_df[book_filter_condition_result]
duration = time.time() - start_time

print(f"book_rating_count_df.shape after book filtering = {book_rating_count_df.shape}")
print(f"\nduration of filtering by books = {duration // 60: .0f}m {duration % 60: .0f}s")

# Check the results
print(f"\nbooks_user_like_set = {books_user_like_set}")
book_rating_count_df[book_rating_count_df.apply(lambda row: row['isbn'] in books_user_like_set, axis=1)]

book_rating_count_df.shape before book filtering = (240478, 2)
book_rating_count_df.shape after book filtering = (52474, 2)

duration of filtering by books =  0m  2s

books_user_like_set = {'0345350499', '0590353403', '0439064872'}


,isbn,book_ratings_no
40705,0345350499,84
76707,0439064872,144
115246,0590353403,81


In [17]:
print(f"ratings_df.shape before filtering by book: {ratings_df.shape}")
ratings_df = ratings_df.merge(book_rating_count_df, on='isbn', how='inner')
ratings_df.drop('book_ratings_no', inplace=True, axis=1)
print(f"ratings_df.shape after filtering by book: {ratings_df.shape}")

ratings_df.shape before filtering by book: (658805, 3)
ratings_df.shape after filtering by book: (431174, 3)


### Find users with similar book preferences as the current user

In [18]:
# Users who read the same books as the current_user
overlap_users = {}

start_time = time.time()

for index, row in ratings_df.iterrows():
    user_id = row['user_id']
    book_isbn = row['isbn']
    if book_isbn in books_user_like_set:
        if user_id not in overlap_users:
            overlap_users[user_id] = 1
        else:
            overlap_users[user_id] += 1 # Track how many books read and rated by current_user have been read and rated by the current user in a loop

duration = time.time() - start_time
print(f"duration of creating the overlap_users = {duration // 60: .0f}m {duration % 60: .0f}s\n")

print(f"len(overlap_users) = {len(overlap_users)}")

duration of creating the overlap_users =  0m  17s

len(overlap_users) = 278


In [23]:
counter = 0
for index, user_id in enumerate(overlap_users):
    print(f"{index + 1}) user_id = {user_id}; similar_books_count = {overlap_users[user_id]}")
    counter += 1
    if counter >= 10:
        break

1) user_id = 277427; similar_books_count = 1
2) user_id = 254; similar_books_count = 3
3) user_id = 2033; similar_books_count = 1
4) user_id = 6251; similar_books_count = 1
5) user_id = 6563; similar_books_count = 1
6) user_id = 7346; similar_books_count = 1
7) user_id = 8245; similar_books_count = 1
8) user_id = 8253; similar_books_count = 1
9) user_id = 8681; similar_books_count = 2
10) user_id = 9747; similar_books_count = 1


In [214]:
""" 
    Filter users, leaving only those who have read all the books of current_user 
        (if len(books_user_like) <= MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF) 
    or MIN_USERS_OVERLAP_COEFF a part of the books read by current_user 
        (if len(books_user_like) > MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF)
"""

start_time = time.time()

filtered_overlap_user_ids_set = None
if len(books_user_like) <= MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: 
    print(f"len(books_user_like) <= MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: {len(books_user_like)} <= {MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF}")
    books_thresh = len(books_user_like)
    filtered_overlap_user_ids_set = set([k for k in overlap_users if overlap_users[k] >= books_thresh])
else:
    print(f"len(books_user_like) > MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: {len(books_user_like)} > {MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF}")
    books_thresh = int(len(books_user_like) * MIN_USERS_OVERLAP_COEFF)
    print(f"books_thresh = {books_thresh}")
    filtered_overlap_user_ids_set = set([k for k in overlap_users if overlap_users[k] > books_thresh])

duration = time.time() - start_time

print(f"duration of filtering users = {duration // 60: .0f}m {duration % 60: .0f}s\n")
print(f"len(filtered_overlap_user_ids_set) = {len(filtered_overlap_user_ids_set)}")

len(books_user_like) > MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: 3 > 2
books_thresh = 1
duration of filtering users =  0m  0s

len(filtered_overlap_user_ids_set) = 27


#### Check the results

In [190]:
# To check the results
print(f"len(filtered_overlap_user_ids_set) = {len(filtered_overlap_user_ids_set)}")
print(f"list(filtered_overlap_user_ids_set)[:10] = {list(filtered_overlap_user_ids_set)[:10]}")

len(filtered_overlap_user_ids_set) = 27
list(filtered_overlap_user_ids_set)[:10] = [148744, 131594, 30735, 240144, 226965, 21014, 11676, 36256, 238120, 100906]


In [191]:
# Check the results
result = (
    ratings_df[['user_id', 'isbn']]
    .apply(lambda row: row['user_id'] in filtered_overlap_user_ids_set and row['isbn'] in books_user_like_set, axis=1)
)

filtered_ratings_df = ratings_df[result]
filtered_ratings_df = (
    filtered_ratings_df
    .groupby('user_id')
    .count()
    .reset_index()
    .rename(columns={'rating': 'ratings_no'})
    .sort_values(by='ratings_no')
)

print(f"len(filtered_ratings_df) = {len(filtered_ratings_df)}")

filtered_ratings_df

len(filtered_ratings_df) = 27


,user_id,isbn,ratings_no
13,98741,2,2
24,235842,2,2
23,229741,2,2
22,226965,2,2
21,225996,2,2
20,211426,2,2
18,150124,2,2
17,148744,2,2
16,131594,2,2
15,110973,2,2


### Find users with similar book preferences as the current user without iterrows()

In [225]:
start_time = time.time()

# Filter ratings_df to only include rows where 'isbn' is in books_user_like_set
filtered_ratings_df = ratings_df[ratings_df['isbn'].isin(books_user_like_set)]

# Group by 'user_id' and count the number of occurrences of each user_id
overlap_users_series = (
    filtered_ratings_df
    .groupby('user_id')
    .count()
    .reset_index()
    .rename(columns={'rating': 'ratings_no'})
)

""" 
    Filter users, leaving only those who have read all the books of current_user 
        (if len(books_user_like) <= MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF) 
    or MIN_USERS_OVERLAP_COEFF a part of the books read by current_user 
        (if len(books_user_like) > MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF)
"""

filtered_overlap_user_ids_set = None
if len(books_user_like) <= MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: 
    print(f"len(books_user_like) <= MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: {len(books_user_like)} <= {MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF}")
    books_thresh = 1
    filtered_overlap_user_ids_set = set(overlap_users_series[overlap_users_series['ratings_no'] >= books_thresh]['user_id'].values)
else:
    print(f"len(books_user_like) > MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: {len(books_user_like)} > {MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF}")
    books_thresh = int(len(books_user_like) * MIN_USERS_OVERLAP_COEFF)
    print(f"books_thresh = {books_thresh}")
    filtered_overlap_user_ids_set = set(overlap_users_series[overlap_users_series['ratings_no'] > books_thresh]['user_id'].values)

duration = time.time() - start_time
print(f"duration of creating the overlap_users and filtering users = {duration // 60: .0f}m {duration % 60: .0f}s\n")

print(f"len(overlap_users_series) = {len(overlap_users_series)}")
print(f"len(filtered_overlap_user_ids_set) = {len(filtered_overlap_user_ids_set)}")

overlap_users_series.head()

len(books_user_like) > MIN_BOOKS_NO_TO_USE_OVERLAP_COEFF: 3 > 2
books_thresh = 1
duration of creating the overlap_users and filtering users =  0m  0s

len(overlap_users_series) = 664
len(filtered_overlap_user_ids_set) = 32


,user_id,isbn,ratings_no
0,254,3,3
1,1725,1,1
2,2033,1,1
3,2179,1,1
4,2337,1,1


#### Check the results

In [42]:
# To check the results
print(f"len(filtered_overlap_user_ids_set) = {len(filtered_overlap_user_ids_set)}")
print(f"list(filtered_overlap_user_ids_set)[:10] = {list(filtered_overlap_user_ids_set)[:10]}")

len(filtered_overlap_user_ids_set) = 168
list(filtered_overlap_user_ids_set)[:10] = [232964, 45064, 131594, 95758, 30735, 137742, 240144, 23571, 20501, 179734]


In [43]:
# Check the results
result = (
    ratings_df[['user_id', 'isbn']]
    .apply(lambda row: row['user_id'] in filtered_overlap_user_ids_set and row['isbn'] in books_user_like_set, axis=1)
)

filtered_ratings_df = ratings_df[result]
filtered_ratings_df = (
    filtered_ratings_df
    .groupby('user_id')
    .count()
    .reset_index()
    .rename(columns={'rating': 'ratings_no'})
    .sort_values(by='ratings_no')
)

print(f"len(filtered_ratings_df) = {len(filtered_ratings_df)}")

filtered_ratings_df

len(filtered_ratings_df) = 168


,user_id,isbn,ratings_no
0,254,1,1
107,177284,1,1
108,177842,1,1
109,178969,1,1
110,179734,1,1
...,...,...,...
58,91425,1,1
59,92494,1,1
60,93047,1,1
52,86495,1,1


## Find similar user book ratings

In [226]:
# For filtered users, user_id, book_isbn and rating will be stored in interactions_arr
start_time = time.time()

interactions_arr = ratings_df[(
    ratings_df['user_id']
    .apply(lambda user_id: user_id in filtered_overlap_user_ids_set) # Maybe also filter: isbn not in books_user_like_set - no need, because it will be used later to find the users with the same book tastes as current_user 
)].values

duration = time.time() - start_time
print(f"duration of creating the interactions_arr = {duration // 60: .0f}m {duration % 60: .0f}s\n")

print(f"len(interactions_arr) = {len(interactions_arr)}")
print(f"interactions_arr[:5] = {interactions_arr[:5]}")

duration of creating the interactions_arr =  0m  0s

len(interactions_arr) = 32207
interactions_arr[:5] = [[254 '006000438X' 0]
 [254 '0060013117' 0]
 [254 '0060199563' 0]
 [254 '0060391448' 0]
 [254 '0060502320' 7]]


In [227]:
interactions_df = pd.DataFrame(data=interactions_arr, columns=['user_id', 'isbn', 'rating'])

# Check the results
print(f"len(interactions_df) = {len(interactions_df)}\n")
print(f"len(filtered_overlap_user_ids_set) = {len(filtered_overlap_user_ids_set)}")
print(f"interactions_df['user_id'].nunique() = {interactions_df['user_id'].nunique()}")

len(interactions_df) = 32207

len(filtered_overlap_user_ids_set) = 32
interactions_df['user_id'].nunique() = 32


In [228]:
interactions_df.head(3)

,user_id,isbn,rating
0,254,006000438X,0
1,254,0060013117,0
2,254,0060199563,0


## Create a user/book matrix

In [229]:
interactions_df = pd.concat([books_user_like_df[['user_id', 'isbn', 'rating']], interactions_df])
interactions_df

,user_id,isbn,rating
0,-1,0590353403,10
1,-1,0439064872,9
2,-1,0345350499,10
0,254,006000438X,0
1,254,0060013117,0
...,...,...,...
32202,272786,0446672211,8
32203,272786,0465017606,0
32204,272786,059035342X,9
32205,272786,0679723161,0


In [230]:
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32210 entries, 0 to 32206
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  32210 non-null  object
 1   isbn     32210 non-null  object
 2   rating   32210 non-null  object
dtypes: object(3)
memory usage: 1006.6+ KB


In [231]:
interactions_df['user_id'] = pd.to_numeric(interactions_df['user_id'])
interactions_df['rating'] = pd.to_numeric(interactions_df['rating'])
interactions_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32210 entries, 0 to 32206
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  32210 non-null  int64 
 1   isbn     32210 non-null  object
 2   rating   32210 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1006.6+ KB


In [232]:
interactions_df['user_index'] = interactions_df['user_id'].astype('category').cat.codes
interactions_df['book_index'] = interactions_df['isbn'].astype('category').cat.codes

# To check the correctness of creating cols 'user_index' and 'book_index'
print(f"interactions_df['user_id'].nunique() = {interactions_df['user_id'].nunique()}")
print(f"interactions_df['user_index'].nunique() = {interactions_df['user_index'].nunique()}\n")

print(f"interactions_df['isbn'].nunique() = {interactions_df['isbn'].nunique()}")
print(f"interactions_df['book_index'].nunique() = {interactions_df['book_index'].nunique()}\n")

interactions_df.head(3)

interactions_df['user_id'].nunique() = 33
interactions_df['user_index'].nunique() = 33

interactions_df['isbn'].nunique() = 25781
interactions_df['book_index'].nunique() = 25781



,user_id,isbn,rating,user_index,book_index
0,-1,0590353403,10,0,14709
1,-1,0439064872,9,0,9368
2,-1,0345350499,10,0,4081


In [233]:
ratings_matrix_coo = coo_matrix((interactions_df['rating'], (interactions_df['user_index'], interactions_df['book_index'])))
ratings_matrix = ratings_matrix_coo.tocsr()

## Find users similar to current_user

In [234]:
# Get user_index for the current_user
current_user_index = interactions_df[interactions_df['user_id'] == -1]['user_index'].unique()[0]
print(f"current_user_index = {current_user_index}")

similarity = cosine_similarity(ratings_matrix[current_user_index, :], ratings_matrix).flatten()
print(f"len(similarity) = {len(similarity)}")

SIMILAR_USERS_NO = (len(similarity) - 1) if len(similarity) < SIMILAR_USERS_NO else SIMILAR_USERS_NO

similar_user_indices = np.argpartition(similarity, -(SIMILAR_USERS_NO + 1))[-(SIMILAR_USERS_NO + 1):]
similar_user_indices = np.delete(similar_user_indices, np.where(similar_user_indices == 0))
print(f"SIMILAR_USERS_NO = {SIMILAR_USERS_NO}; len(similar_user_indices) = {len(similar_user_indices)}")

similar_users = interactions_df[interactions_df['user_index'].isin(similar_user_indices)].copy()
print(f"similar_users['user_id'].nunique() = {similar_users['user_id'].nunique()}\n")
similar_users

current_user_index = 0
len(similarity) = 33
SIMILAR_USERS_NO = 15; len(similar_user_indices) = 15
similar_users['user_id'].nunique() = 15



,user_id,isbn,rating,user_index,book_index
0,254,006000438X,0,1,168
1,254,0060013117,0,1,195
2,254,0060199563,0,1,384
3,254,0060391448,0,1,451
4,254,0060502320,7,1,475
...,...,...,...,...,...
32202,272786,0446672211,8,32,10926
32203,272786,0465017606,0,32,12441
32204,272786,059035342X,9,32,14710
32205,272786,0679723161,0,32,16903


## Create book recommendations using the 'book_score' variable

#### Create book_recs_df dataframe

In [235]:
book_recs_df = (
    similar_users
    .groupby('isbn')
    ['rating']
    .agg(['count', 'mean'])
    .rename(columns = {
        'count': 'ratings_no',
        'mean': 'avg_rating'
    })
    .sort_values(by='ratings_no', ascending=False)
)

print(f"book_recs_df.shape = {book_recs_df.shape}")
book_recs_df

book_recs_df.shape = (3147, 2)


,ratings_no,avg_rating
isbn,,
0590353403,14,6.714286
0345350499,10,8.400000
0439136350,10,7.100000
0439064864,10,6.300000
0439064872,9,8.111111
...,...,...
0425097250,1,0.000000
0425097722,1,0.000000
0425117693,1,0.000000


In [236]:
# merge book_recs_df and books_df to get all the information about recommended books
book_recs_df = book_recs_df.merge(books_df, how='inner', on='isbn')

print(f"book_recs_df.shape before filtering = {book_recs_df.shape}")
if len(books_user_like_set) >= MIN_BOOK_RATINGS_NO_CFB:
    book_recs_df = book_recs_df[book_recs_df['ratings_no'] >= MIN_BOOK_RATINGS_NO_CFB]
print(f"book_recs_df.shape after filtering = {book_recs_df.shape}")

book_recs_df.shape before filtering = (3147, 8)
book_recs_df.shape after filtering = (46, 8)


In [237]:
book_total_ratings_df = (
    ratings_df
    .groupby('isbn')
    ['rating']
    .count()
    .reset_index()
    .rename(columns = {'rating': 'total_ratings_no'})
)
print(f"book_total_ratings_df.shape = {book_total_ratings_df.shape}\n")
book_total_ratings_df.sort_values(by='total_ratings_no', ascending=False).head()

book_total_ratings_df.shape = (270150, 2)



,isbn,total_ratings_no
215951,0971880107,2502
38570,0316666343,1295
70798,0385504209,883
7344,0060928336,732
32370,0312195516,723


In [238]:
book_recs_df = book_recs_df.merge(book_total_ratings_df, how='inner', on='isbn')
print(f"book_recs_df.shape = {book_recs_df.shape}\n")
book_recs_df.head(3)

book_recs_df.shape = (46, 9)



,isbn,ratings_no,avg_rating,title,author,publication_year,publisher,image_url,total_ratings_no
0,0590353403,14,6.714286,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...,168
1,0345350499,10,8.400000,The Mists of Avalon,MARION ZIMMER BRADLEY,1987,Del Rey,http://images.amazon.com/images/P/0345350499.0...,181
2,0439136350,10,7.100000,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,197


In [239]:
# Get the current columns
cols = book_recs_df.columns.tolist()

# Rearrange columns to move the last column to the 4th position
new_cols = cols[:3] + [cols[-1]] + cols[3:-1]
print(f"new_cols = {new_cols}")

# Reindex the DataFrame with the new column order
book_recs_df = book_recs_df.reindex(columns=new_cols)
book_recs_df.head(3)

new_cols = ['isbn', 'ratings_no', 'avg_rating', 'total_ratings_no', 'title', 'author', 'publication_year', 'publisher', 'image_url']


,isbn,ratings_no,avg_rating,total_ratings_no,title,author,publication_year,publisher,image_url
0,0590353403,14,6.714286,168,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...
1,0345350499,10,8.400000,181,The Mists of Avalon,MARION ZIMMER BRADLEY,1987,Del Rey,http://images.amazon.com/images/P/0345350499.0...
2,0439136350,10,7.100000,197,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...


#### Create the 'book_score' variable to recommend books

In [240]:
book_recs_df['adjusted_ratings_no'] = book_recs_df['ratings_no'] * (book_recs_df['ratings_no'] / book_recs_df['total_ratings_no'])
book_recs_df['book_score'] = book_recs_df['avg_rating'] * book_recs_df['adjusted_ratings_no']

# Get the current columns
cols = book_recs_df.columns.tolist()

# Rearrange columns to move the last column to the 4th position
new_cols = cols[:4] + cols[-2:] + cols[4:-2]

# Reindex the DataFrame with the new column order
book_recs_df = book_recs_df.reindex(columns=new_cols)

book_recs_df.head(3)

,isbn,ratings_no,avg_rating,total_ratings_no,adjusted_ratings_no,book_score,title,author,publication_year,publisher,image_url
0,0590353403,14,6.714286,168,1.166667,7.833333,Harry Potter and the Sorcerer's Stone (Book 1),J. K. Rowling,1998,Scholastic,http://images.amazon.com/images/P/0590353403.0...
1,0345350499,10,8.400000,181,0.552486,4.640884,The Mists of Avalon,MARION ZIMMER BRADLEY,1987,Del Rey,http://images.amazon.com/images/P/0345350499.0...
2,0439136350,10,7.100000,197,0.507614,3.604061,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...


#### Filter the results and recommend books

In [241]:
# Filter book_recs_df
print(f"book_recs_df.shape before filtering = {book_recs_df.shape}")

book_recs_df = book_recs_df[~book_recs_df['isbn'].isin(books_user_like_df['isbn'])]

# Create a new 'mod_title' column in both book_recs_df and books_user_like_df 
book_recs_df['mod_title'] = book_recs_df['title'].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()
book_recs_df['mod_title'] = book_recs_df['mod_title'].str.replace("\s+", " ", regex=True)

books_user_like_df['mod_title'] = books_user_like_df['title'].str.replace("[^a-zA-Z0-9 ]", "", regex=True).str.lower()
books_user_like_df['mod_title'] = books_user_like_df['mod_title'].str.replace("\s+", " ", regex=True)

book_recs_df = book_recs_df[~book_recs_df['mod_title'].isin(books_user_like_df['mod_title'])]

print(f"book_recs_df.shape after filtering = {book_recs_df.shape}")

book_recs_df.shape before filtering = (46, 11)
book_recs_df.shape after filtering = (42, 12)


In [242]:
top_books_recs_df = book_recs_df.drop_duplicates('title').sort_values(by=['book_score', 'total_ratings_no'], ascending=[False, False])
top_books_recs_df.head(10)

,isbn,ratings_no,avg_rating,total_ratings_no,adjusted_ratings_no,book_score,title,author,publication_year,publisher,image_url,mod_title
2,0439136350,10,7.100000,197,0.507614,3.604061,Harry Potter and the Prisoner of Azkaban (Book 3),J. K. Rowling,1999,Scholastic,http://images.amazon.com/images/P/0439136350.0...,harry potter and the prisoner of azkaban book 3
30,0451458621,3,5.333333,20,0.450000,2.400000,Priestess Of Avalon,Marion Zimmer Bradley,2002,Roc,http://images.amazon.com/images/P/0451458621.0...,priestess of avalon
20,0618002219,4,8.750000,70,0.228571,2.000000,The Hobbit: or There and Back Again,J.R.R. Tolkien,1999,Houghton Mifflin Company,http://images.amazon.com/images/P/0618002219.0...,the hobbit or there and back again
6,0439139597,7,6.714286,194,0.252577,1.695876,Harry Potter and the Goblet of Fire (Book 4),J. K. Rowling,2000,Scholastic,http://images.amazon.com/images/P/0439139597.0...,harry potter and the goblet of fire book 4
17,0451454243,4,6.250000,65,0.246154,1.538462,The Forest House,Marion Zimmer Bradley,1995,Roc,http://images.amazon.com/images/P/0451454243.0...,the forest house
22,0618002227,3,8.333333,63,0.142857,1.190476,The Fellowship of the Ring (The Lord of the Ri...,J. R. R. Tolkien,1999,Houghton Mifflin Company,http://images.amazon.com/images/P/0618002227.0...,the fellowship of the ring the lord of the rin...
5,043935806X,7,6.857143,334,0.146707,1.005988,Harry Potter and the Order of the Phoenix (Boo...,J. K. Rowling,2003,Scholastic,http://images.amazon.com/images/P/043935806X.0...,harry potter and the order of the phoenix book 5
38,0385425074,3,3.333333,33,0.272727,0.909091,Backlash: The Undeclared War Against American ...,Susan Faludi,1992,Anchor,http://images.amazon.com/images/P/0385425074.0...,backlash the undeclared war against american w...
45,1573225517,3,9.000000,105,0.085714,0.771429,High Fidelity,Nick Hornby,1996,Riverhead Books,http://images.amazon.com/images/P/1573225517.0...,high fidelity
32,0345325818,3,4.000000,63,0.142857,0.571429,The Silmarillion,J.R.R. TOLKIEN,1985,Del Rey,http://images.amazon.com/images/P/0345325818.0...,the silmarillion
